# CS 176 Exercise 2 (Parameter Tuning)

Instructions: Use Jupyter Notebook and the DEAP package to perform the following exercise. Rename this file with your lastname and submit it via UVLE.


# Part I. Genetic Algorithm
Choose one of the following single objective continuous optimization benchmark problem from the DEAP package:

*   Ackley
*   Bohachevsky
*   Griewank
*   Rosenbrock
*   Schaffer
*   Schwefel

The details of these problems can be found here:

https://deap.readthedocs.io/en/master/api/benchmarks.html#continuous-optimization


Use Genetic Algorithm (GA) to solve it.  Use the appropriate representation.

Select 3 different values for each of the following parameters:
* population size
* mutation rate
* crossover rate


# **Fill in the following details here:**

1.   Benchmark problem: Rosenbrock
2.   Representation: list of floats
3.   Population size values: 20, 50, 100 
4.   Mutation rate values: 0.2, 0.4, 0.6
5.   Crossover rate values: 0.3, 0.5, 0.7
      

# Genetic Algorithm Implementation

Implement Genetic Algorithm using the DEAP package. Execute 30 runs with each of the 27 different GA instances and for each run save the following information:

*	best fitness at termination
*	number of fitness evaluations
*	CPU time needed to complete the run

Perform a simple statistical analysis on the spread of the outcomes, calculate the following:
*	minimum
*	maximum
*	average
*	standard deviation





In [3]:
import random
import time
from deap import tools, base, creator, benchmarks

#Initializations
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

IND_SIZE = 2
POPULATION_SIZE = 20

toolbox = base.Toolbox()
toolbox.register("attribute", random.uniform, a=-2.00, b=2.00)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attribute, n=IND_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evaluate(individual):
    return benchmarks.rosenbrock(individual)

#Genetic Algorithm Specifications
toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.5)
toolbox.register("select", tools.selRoulette)
toolbox.register("evaluate", evaluate)

def main():
    pop = toolbox.population(n=POPULATION_SIZE)
    CXPB, MUTPB, NGEN = 0.3, 0.2, 50
    fitness_evals = POPULATION_SIZE

    # Evaluate the entire population
    fitnesses = map(toolbox.evaluate, pop)
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    for g in range(NGEN):
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # The population is entirely replaced by the offspring
        pop[:] = offspring
        fitness_evals += len(invalid_ind)

    return pop, fitness_evals

POPULATION, FITNESS_EVALS = main()
MAX_FITNESS = max([i[0] for i in list(map(evaluate, POPULATION))])

print(f'CPU Time = {time.process_time()}')
print(f'Best Fitness = {MAX_FITNESS}')
print(f'Fitness Evalutations = {FITNESS_EVALS}')



CPU Time = 1.5
Best Fitness = 3251.947215353377
Fitness Evalutations = 484


# Analysis

Compare the results for the 27 GA instances and try to draw conclusions about good parameter values. Are the good parameter values for the highest solution quality and the ones for the fastest GA the same?

To analyze the provided data, we will consider the different combinations of population size, mutation rate, and crossover rate. We will examine how changes in these parameters affect the CPU time, fitness value, and evaluations of the algorithm. Let's break down the analysis step by step:

-Effect of Mutation and Crossover Rates on CPU Time:

Comparing the data points, it appears that as the mutation and crossover rates increase, the CPU time generally increases as well. This indicates that higher mutation and crossover rates might lead to more complex computations.

-Impact of Population Size on CPU Time and Evaluations:

With an increase in the population size, the CPU time and evaluations tend to rise as well. This is expected as larger populations require more computational resources and time for evaluation.

-Influence of Parameters on Fitness Value:

The trend in fitness values is not as straightforward as the other metrics. It seems that changing the mutation and crossover rates does not consistently affect the fitness value. However, higher population sizes seem to lead to better fitness values in general.

-Effect of Parameters on Standard Deviation:

The standard deviation for the metrics varies across different parameter configurations. Higher mutation and crossover rates seem to result in larger standard deviations, indicating greater variability in results. Similarly, larger population sizes also lead to increased standard deviations.

-Optimal Configurations:

Based on the data provided, the configurations that generally exhibit better performance, i.e., lower CPU time and better fitness values, seem to be when the population size is larger (50 or 100) and the mutation and crossover rates are moderate (0.4 or 0.5). These configurations strike a balance between computational complexity and solution quality.

It is important to note that the analysis provided here is based on the limited data and assumptions made from this data set alone. For a more comprehensive understanding, additional experiments and statistical tests would be required.

RAW DATA:

the text below is formatted in the following format:

POPULATION = [pop_size], MUTATION_RATE = [mutate_rate],  CROSSOVER_RATE[crossover_rate]
[min CPU TIME], [max CPU TIME], [mean CPU TIME], [std deviation CPU TIME]
[min FITNESS VALUE], [max FITNESS VALUE], [mean FITNESS VALUE], [std deviation FITNESS]
[min EVALUATIONS], [max EVALUATIONS], [mean EVALUATIONS], [std deviation EVALUATIONS]

POPULATION = 20, MUTATION_RATE = 0.2, CROSSOVER_RATE = 0.3
0.140625, 0.5625, 0.35104166666666664, 0.12301373286028769
1867.4112796591678, 3518.818421270459, 2741.6318498389996, 463.46314079737357
431, 503, 471.23333333333335, 18.08808693280992

POPULATION = 20, MUTATION_RATE = 0.2, CROSSOVER_RATE = 0.5
0.1875, 0.609375, 0.4005208333333333, 0.12931883677615913
1883.3496362048415, 3468.180012867922, 2794.0400795768805, 403.51873241885926
565, 665, 617.7333333333333, 24.329588752975027

POPULATION = 20, MUTATION_RATE = 0.2, CROSSOVER_RATE = 0.7
0.25, 0.703125, 0.4765625, 0.13524127263123734
1361.681062453152, 3540.304996442589, 2736.544423684072, 494.52238695530724
740, 812, 775.3333333333334, 17.059373441666086

POPULATION = 20, MUTATION_RATE = 0.4, CROSSOVER_RATE = 0.3
0.21875, 0.6875, 0.4473958333333333, 0.13843687296070686
834.5254725513621, 3572.48042765257, 2533.858968425452, 640.2076773486272
563, 629, 597.2666666666667, 16.303237578945954

POPULATION = 20, MUTATION_RATE = 0.4, CROSSOVER_RATE = 0.5
0.390625, 1.046875, 0.75, 0.19875098270197308
1415.9523083391844, 3451.959612032185, 2641.3692531343327, 520.1000000521244
681, 777, 724.1666666666666, 21.32773364008052

POPULATION = 20, MUTATION_RATE = 0.4, CROSSOVER_RATE = 0.7
0.21875, 0.671875, 0.4453125, 0.13524127263123734
1099.6541139871817, 3367.5394949562847, 2630.371769112229, 519.5734747996876
808, 862, 834.6, 13.116401945655676

POPULATION = 20, MUTATION_RATE = 0.6, CROSSOVER_RATE = 0.3
0.171875, 0.6875, 0.42604166666666665, 0.1518083987484077
1297.8143714038201, 3509.4311294370173, 2251.096797872333, 661.3155167424303
716, 761, 738.5666666666667, 13.763922244605844

POPULATION = 20, MUTATION_RATE = 0.6, CROSSOVER_RATE = 0.5
0.21875, 0.6875, 0.45416666666666666, 0.14194595631397497
1129.3536988616445, 3443.1790676310943, 2177.708871620458, 692.2046909927669
772, 859, 817.2666666666667, 16.09540997372301

POPULATION = 20, MUTATION_RATE = 0.6, CROSSOVER_RATE = 0.7
0.203125, 0.6875, 0.4453125, 0.14589377467727446
1045.6889241766519, 3547.2342791207147, 2483.5853344898414, 600.6848269965011
887, 922, 902.5333333333333, 9.548938277223401

POPULATION = 50, MUTATION_RATE = 0.2, CROSSOVER_RATE = 0.3
0.25, 1.78125, 1.0322916666666666, 0.4564521051606133
2208.974117639252, 3503.7969231794373, 3067.887639093219, 308.4090601397354
1098, 1217, 1152.1, 27.19601686522005

POPULATION = 50, MUTATION_RATE = 0.2, CROSSOVER_RATE = 0.5
0.203125, 1.78125, 1.0015625, 0.47154289499781105
1897.4157527804773, 3606.9252472689977, 3103.5349351007076, 294.6568127550632
1483, 1599, 1546.5333333333333, 27.718505651559855

POPULATION = 50, MUTATION_RATE = 0.2, CROSSOVER_RATE = 0.7
0.21875, 1.9375, 1.0979166666666667, 0.5252717137481219
2487.8445966211825, 3526.8450666756353, 3192.384828287702, 268.6354972917967
1885, 2011, 1945.2, 28.68960322718551

POPULATION = 50, MUTATION_RATE = 0.4, CROSSOVER_RATE = 0.3
0.234375, 1.953125, 1.0958333333333334, 0.5107205727912497
2445.3550404987177, 3536.4181579258297, 3005.209568951179, 310.1325898813823
1438, 1536, 1491.1666666666667, 26.236213310782652

POPULATION = 50, MUTATION_RATE = 0.4, CROSSOVER_RATE = 0.5
0.28125, 1.921875, 1.0994791666666666, 0.48753143817789385
2298.2300615801682, 3546.797196610683, 3077.384259816037, 309.67334698225267
1766, 1852, 1799.4333333333334, 19.462242647980958

POPULATION = 50, MUTATION_RATE = 0.4, CROSSOVER_RATE = 0.7
0.234375, 1.9375, 1.0973958333333333, 0.507490816915412
2372.5679692394556, 3498.274273302835, 3086.818350484505, 277.5100165569537
2031, 2134, 2094.133333333333, 23.19588086037883

POPULATION = 50, MUTATION_RATE = 0.6, CROSSOVER_RATE = 0.3
0.234375, 2.0, 1.1161458333333334, 0.5240579862987227
1927.1772333702052, 3488.7903542925474, 2910.0780009754, 422.3698969464999
1797, 1897, 1841.7333333333333, 26.076724402339256

POPULATION = 50, MUTATION_RATE = 0.6, CROSSOVER_RATE = 0.5
0.25, 2.0, 1.1291666666666667, 0.5175930455134505
2016.8798830927528, 3549.1386085825575, 2943.563600369968, 321.8721702314187
1951, 2086, 2045.5, 25.813110364050797

POPULATION = 50, MUTATION_RATE = 0.6, CROSSOVER_RATE = 0.7
0.265625, 2.03125, 1.1578125, 0.5285216397631384
2180.5051073407285, 3516.7228318696934, 3081.6256653139526, 378.2629633842059
2213, 2279, 2246.8333333333335, 18.279466318492148

POPULATION = 100, MUTATION_RATE = 0.2, CROSSOVER_RATE = 0.3
0.375, 5.671875, 3.043229166666667, 1.5803598209358753
2947.5125496285364, 3557.066837519937, 3316.876567316494, 147.692623261856
2158, 2387, 2291.9, 52.24707328326312

POPULATION = 100, MUTATION_RATE = 0.2, CROSSOVER_RATE = 0.5
0.46875, 6.203125, 3.3203125, 1.7306677251106242
2905.200674674129, 3581.5208568077405, 3319.142155039823, 149.73523907566693
3011, 3155, 3093.6666666666665, 36.061367448035334

POPULATION = 100, MUTATION_RATE = 0.2, CROSSOVER_RATE = 0.7
0.421875, 6.3125, 3.3447916666666666, 1.7699427907813543
2892.8206120994973, 3577.758743518878, 3365.2819469239084, 163.53064578484464
3845, 3997, 3909.9, 32.81904934637809

POPULATION = 100, MUTATION_RATE = 0.4, CROSSOVER_RATE = 0.3
0.484375, 7.046875, 3.9380208333333333, 1.9252150994768018
2815.967515761501, 3563.524001922004, 3299.4052140111744, 186.16415821218266
2852, 3087, 2993.4333333333334, 45.0293854672208

POPULATION = 100, MUTATION_RATE = 0.4, CROSSOVER_RATE = 0.5
0.546875, 7.1875, 3.754166666666667, 1.9324149219449624
2853.080140063572, 3568.917634689584, 3352.575161479469, 171.9022946912375
3517, 3708, 3583.1, 40.801429713512086

POPULATION = 100, MUTATION_RATE = 0.4, CROSSOVER_RATE = 0.7
0.796875, 7.109375, 3.995833333333333, 1.8469469693629237
3038.209679767145, 3549.710731643602, 3304.4303674878392, 137.38527239160663
4126, 4251, 4191.733333333334, 31.891412985664687

POPULATION = 100, MUTATION_RATE = 0.6, CROSSOVER_RATE = 0.3
0.515625, 6.953125, 3.7333333333333334, 1.9244644366824646
2627.5408607786167, 3571.991004201504, 3278.070597842835, 215.5073505717283
3625, 3775, 3691.7, 36.58793061470772

POPULATION = 100, MUTATION_RATE = 0.6, CROSSOVER_RATE = 0.5
0.515625, 6.984375, 3.75625, 1.928189153443804
2815.996019376687, 3594.1993245106432, 3315.555237233186, 181.80143496828643
4039, 4157, 4102.366666666667, 25.857923780192063

POPULATION = 100, MUTATION_RATE = 0.6, CROSSOVER_RATE = 0.7
0.546875, 7.078125, 3.8208333333333333, 1.947866810859311
3093.7370621813643, 3562.662450277206, 3342.6363002877047, 125.62827407751288
4468, 4573, 4499.0, 24.664414311581236

# Further Parameter Tuning
Fix one parameter at one of its values and do the same analysis for the 9 corresponding runs.

In [ ]:
import random
import time
from deap import tools, base, creator, benchmarks

#Initializations
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

IND_SIZE = 2
POPULATION_SIZE = 50

toolbox = base.Toolbox()
toolbox.register("attribute", random.uniform, a=-2.00, b=2.00)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attribute, n=IND_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evaluate(individual):
    return benchmarks.rosenbrock(individual)

#Genetic Algorithm Specifications
toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.5)
toolbox.register("select", tools.selRoulette)
toolbox.register("evaluate", evaluate)

def main():
    pop = toolbox.population(n=POPULATION_SIZE)
    CXPB, MUTPB, NGEN = 0.3, 0.2, 50
    fitness_evals = POPULATION_SIZE

    # Evaluate the entire population
    fitnesses = map(toolbox.evaluate, pop)
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    for g in range(NGEN):
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # The population is entirely replaced by the offspring
        pop[:] = offspring
        fitness_evals += len(invalid_ind)

    return pop, fitness_evals

POPULATION, FITNESS_EVALS = main()
MAX_FITNESS = max([i[0] for i in list(map(evaluate, POPULATION))])

print(f'CPU Time = {time.process_time()}')
print(f'Best Fitness = {MAX_FITNESS}')
print(f'Fitness Evalutations = {FITNESS_EVALS}')



# Analysis
How does this change your results? Summarise your observations in a short report.

Since it seems that Population size has the most significant and direct effect to CPU time, Fitness values, and Evaluations, I have chosen to tune this value by fixing it to 50. Not too much to the point that it dows not consume too much compute power and not to small as to lessen fitness.

Fixing the population size in the data set allowed for a more direct comparison of the effects of different mutation and crossover rates on the algorithm's performance. This facilitated a clearer understanding of the impact of these parameters on the computational metrics. Specifically, the fixation of the population size to 50 in the data set highlighted the following:

-Consistency in Computational Resources:

With a fixed population size, the comparison of CPU times across different mutation and crossover rates became more straightforward. The consistency in population size enabled a clearer observation of how changes in mutation and crossover rates affected the computational workload, as variations in CPU time were more likely to be due to these parameter changes rather than population size differences.

-Direct Comparison of Evaluation Counts:

Having a fixed population size allowed for a more direct comparison of the number of evaluations required for each configuration. This highlighted how changes in mutation and crossover rates influenced the number of evaluations needed to reach optimal solutions. The fixed population size facilitated a clearer understanding of how algorithmic adjustments influenced the efficiency of the optimization process.

-Improved Comparison of Variability:

Fixing the population size also improved the comparison of standard deviations across different configurations. This helped to emphasize the effects of changes in mutation and crossover rates on the variability in the results. By eliminating the influence of varying population sizes, it became more evident that higher mutation and crossover rates tend to introduce more variability, emphasizing the importance of stability and consistency in the algorithm's performance.

By fixing the population size, the focus shifted to isolating the effects of mutation and crossover rates on the algorithm's behavior, providing valuable insights into the impact of these parameters on the overall performance of the genetic algorithm.

RAW DATA:
POPULATION = 50, MUTATION_RATE = 0.2, CROSSOVER_RATE = 0.3
0.3125, 2.296875, 1.2989583333333334, 0.6087167028310917
2284.7400099201923, 3485.411043555861, 3008.121691610035, 318.0640661105592
1081, 1198, 1160.4, 24.8496814198224

POPULATION = 50, MUTATION_RATE = 0.2, CROSSOVER_RATE = 0.5
0.265625, 1.984375, 1.1375, 0.5120043334777549
2660.9612352239983, 3522.326824911597, 3150.5873490435856, 228.41620287018543
1482, 1613, 1566.0, 29.066590213966734

POPULATION = 50, MUTATION_RATE = 0.2, CROSSOVER_RATE = 0.7
0.28125, 2.046875, 1.1744791666666667, 0.5272534001819924
2538.824553733938, 3541.813220261007, 3098.393416490406, 275.82445851507475
1896, 2003, 1950.0333333333333, 26.883059515034535

POPULATION = 50, MUTATION_RATE = 0.4, CROSSOVER_RATE = 0.3
0.265625, 2.125, 1.1963541666666666, 0.554231522285713
2362.8219818988964, 3401.445469623987, 3011.3852613679223, 300.10752415780956
1431, 1544, 1493.6, 24.30994309605297

POPULATION = 50, MUTATION_RATE = 0.4, CROSSOVER_RATE = 0.5
0.265625, 2.15625, 1.2359375, 0.558191689511393
2004.8833761086182, 3569.157644228745, 3136.2060205024804, 378.96208353680174
1763, 1880, 1805.7333333333333, 26.470024975851878

POPULATION = 50, MUTATION_RATE = 0.4, CROSSOVER_RATE = 0.7
0.34375, 2.3125, 1.3479166666666667, 0.5795861790009221
2754.816409067427, 3577.6511722983473, 3222.6174367290646, 211.76039391965276
2053, 2140, 2099.766666666667, 20.58589052941089

POPULATION = 50, MUTATION_RATE = 0.6, CROSSOVER_RATE = 0.3
0.34375, 2.65625, 1.5036458333333333, 0.7036882389146528
1695.913320897551, 3566.8106851544308, 2905.623149993908, 407.16934920140716
1765, 1884, 1849.9, 22.977307646168352

POPULATION = 50, MUTATION_RATE = 0.6, CROSSOVER_RATE = 0.5
0.359375, 2.390625, 1.3880208333333333, 0.5971133268373815
2429.8231655377076, 3542.5728437659563, 3101.460275924487, 256.70553569241736
2016, 2087, 2052.866666666667, 16.126031405429202

POPULATION = 50, MUTATION_RATE = 0.6, CROSSOVER_RATE = 0.7
0.328125, 2.34375, 1.3541666666666667, 0.6010967262475778
2116.6084322656684, 3506.0611313024706, 3043.4742040621204, 279.56596293380284
2223, 2275, 2248.1, 14.709067498202144